In [12]:
import os

In [1]:
import pandas as pd
import networkx as nx

In [2]:
from iac_sketch import data, etl, sketch, transform, system_tests
from iac_sketch.extract import extract_python, extract_yaml

In [26]:
# DEBUG
import importlib

importlib.reload(extract_yaml)
importlib.reload(extract_python)
importlib.reload(etl)
importlib.reload(data)
importlib.reload(transform)
importlib.reload(system_tests)
importlib.reload(sketch)

<module 'iac_sketch.sketch' from '/Users/zhafen/repos/iac-sketch/iac_sketch/sketch.py'>

In [28]:
architect = sketch.Architect(root_dir=os.path.abspath(".."))
registry = architect.perform_registry_etl()
tests, test_results = architect.validate_registry()

In [34]:
registry.view(["test", "FunctionDef"])

,test.comp_key,test.value,FunctionDef.name,FunctionDef.args,FunctionDef.body,FunctionDef.decorator_list,FunctionDef.returns,FunctionDef.type_comment,FunctionDef.type_params,FunctionDef.docstring
entity,,,,,,,,,,
iac_sketch.system_tests.test_connected,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iac_sketch.system_tests.test_defined,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iac_sketch.system_tests.test_designed,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iac_sketch.system_tests.test_implemented,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iac_sketch.system_tests.test_no_forbidden_components,0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
test_executes_tests,1,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
test_perform_registry_etl,1,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
registry.view(["test", "code", "satisfies", "FunctionDef"])

,test.comp_key,test.value,code.value,satisfies.value,FunctionDef.name,FunctionDef.args,FunctionDef.body,FunctionDef.decorator_list,FunctionDef.returns,FunctionDef.type_comment,FunctionDef.type_params,FunctionDef.docstring
entity,,,,,,,,,,,,
iac_sketch.system_tests.test_connected,0,<NA>,NaN,fully_connected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iac_sketch.system_tests.test_defined,0,<NA>,NaN,fully_defined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iac_sketch.system_tests.test_designed,0,<NA>,NaN,fully_designed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iac_sketch.system_tests.test_implemented,0,<NA>,NaN,fully_implemented,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iac_sketch.system_tests.test_no_forbidden_components,0,<NA>,NaN,no_forbidden_components,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
test_executes_tests,1,<NA>,NaN,can_execute_tests,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
test_perform_registry_etl,1,<NA>,NaN,can_perform_registry_etl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
os.path.abspath(architect.root_dir)

'/Users/zhafen/repos/iac-sketch'

In [24]:
architect.extract_sys.filenames

['/Users/zhafen/repos/iac-sketch/base_manifest/requirements.yaml',
 '/Users/zhafen/repos/iac-sketch/base_manifest/relationships.yaml',
 '/Users/zhafen/repos/iac-sketch/base_manifest/components.yaml',
 '/Users/zhafen/repos/iac-sketch/base_manifest/python.yaml',
 '/Users/zhafen/repos/iac-sketch/base_manifest/system.yaml',
 '/Users/zhafen/repos/iac-sketch/base_manifest/objects.yaml',
 '/Users/zhafen/repos/iac-sketch/iac_sketch/extract/__init__.py',
 '/Users/zhafen/repos/iac-sketch/iac_sketch/extract/extract_python.py',
 '/Users/zhafen/repos/iac-sketch/iac_sketch/extract/extract_yaml.py']

In [5]:
registry.view(["test", "code", "satisfies"])

,test.comp_key,test.value,code.value,satisfies.value
entity,,,,
test_executes_tests,1,<NA>,NaN,can_execute_tests
test_perform_registry_etl,1,<NA>,NaN,can_perform_registry_etl


In [ ]:
# Visualize the graph
nx.draw(registry.graph, with_labels=True)

In [ ]:
import ast

In [ ]:
with open("../iac_sketch/etl.py", "r") as f:
    source = f.read()
tree = ast.parse(source)
tree

In [ ]:
list(ast.iter_child_nodes(tree))

In [ ]:
import ast

In [ ]:
node = ast.parse("""
def my_function(x) -> int:
    return x + 1

x = my_function(5)
""")

In [ ]:
nt = ast.NodeTransformer()
nt.visit(node)

In [ ]:
nt = ast.NodeVisitor()
nt.visit(node)

In [ ]:
list(ast.iter_child_nodes(node.body[1]))

In [ ]:
node.body[1].value.func.ctx

In [ ]:
node.body[0].__class__.__name__

In [ ]:
node.body[0]._fields

In [ ]:
node.body[0].body[0].value._fields

In [ ]:
visitor = ComponentVisitor()

In [ ]:
visitor.visit(tree)

In [ ]:
pd.DataFrame(visitor.components)

In [ ]:
type(tree.body[-1])

In [ ]:
dict(ast.iter_fields(child_node))

In [ ]:
components = []
for child_node in ast.iter_child_nodes(tree.body[-1]):
    comp_i = {
        "component_type": type(child_node).__name__
    }
    components.append(comp_i)

components

In [ ]:
tree.body[-1].__dir__()

In [ ]:
for item in ast.iter_fields(tree.body[-1].body[-1]):
    print(item)

In [ ]:
visitor.visit(tree.body[-1])

In [ ]:
ast.walk(visitor.visit(tree))

In [ ]:
for item in ast.walk(tree):
    print(item)

In [ ]:
visitor.visit_Constant(tree)

In [ ]:
components = {}
for field, value in ast.iter_fields(tree):
    print(field, value)
    if isinstance(value, str)

In [ ]:
for item in ast.iter_child_nodes(transform_sys_tree):
    print(item)

In [ ]:
ast.get_source_segment(source, transform_sys_tree)

In [ ]:
ast.get_docstring(transform_sys_tree)

In [ ]:
isinstance(transform_sys_tree, ast.AST)